In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
import seaborn as sns

In [8]:
df=pd.read_csv('diabetes.csv')

In [9]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [10]:
df['time'].value_counts()

KeyError: 'time'

In [ ]:
df.head()

In [ ]:
df.info()

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
encoder=LabelEncoder()

In [13]:
df['time']=encoder.fit_transform(df['time'])

KeyError: 'time'

In [14]:
encoder.classes_

AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [15]:
x=df.drop('Outcome',axis=1)
y=df['Outcome']

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=.8,random_state=21)

In [18]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [20]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [21]:
cat=['sex','smoker','day']
num=['total_bill','tip','size']

In [22]:
cat=[]
num=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']

In [23]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [24]:
#Feature Engineering Automation
num_pipeline=Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler()),
            
        ])
cat_pipeline=Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')),
            ('onehotencoder',OneHotEncoder()),
            
        ])

In [25]:
preprocessor=ColumnTransformer([
    ('num_pipe',num_pipeline,num),
    ('cat_pipe',cat_pipeline,cat)

])

In [26]:
x_train_scaled=preprocessor.fit_transform(x_train)
x_test_scaled=preprocessor.fit_transform(x_test)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [28]:
##Automating Model
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'Naives':GaussianNB(),
    'LogisticRegression':LogisticRegression(),
    'SVC':SVC()
}

In [29]:
len(models)

5

In [30]:
list(models.values())[0]

RandomForestClassifier()

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
def evaluate_model(x_train_scaled,y_train,x_test_scaled,y_test,models):
    reports={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(x_train_scaled,y_train)
        
        y_test_pred=model.predict(x_test_scaled)
        
        test_model_score=accuracy_score(y_test_pred,y_test)
        
        reports[list(models.values())[i]]=round(test_model_score*100,2)
    return reports

In [33]:
evaluate_model(x_train_scaled,y_train,x_test_scaled,y_test,models)

{RandomForestClassifier(): 74.03,
 DecisionTreeClassifier(): 69.48,
 GaussianNB(): 74.03,
 LogisticRegression(): 75.32,
 SVC(): 72.73}

In [34]:
!pip install Xgboost

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xb

In [38]:
clss=AdaBoostClassifier()
clss.fit(x_train_scaled,y_train)

AdaBoostClassifier()

In [44]:
y_pred=clss.predict(x_test_scaled)

In [51]:
from sklearn.metrics import confusion_matrix , accuracy_score, classification_report,r2_score

In [52]:
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test)*100)
print(r2_score(y_pred,y_test)*100)


              precision    recall  f1-score   support

           0       0.87      0.73      0.79       113
           1       0.48      0.71      0.57        41

    accuracy                           0.72       154
   macro avg       0.68      0.72      0.68       154
weighted avg       0.77      0.72      0.73       154

[[82 31]
 [12 29]]
72.07792207792207
-42.931146125620565


In [47]:
Precision=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[0][1]),2)
Accuracy=round((confusion_matrix(y_pred,y_test)[0][0]
          +confusion_matrix(y_pred,y_test)[0][1])*100/sum(sum(confusion_matrix(y_pred,y_test))),2)
Recall=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[1][0]),2)

F1_Score=round(2*Precision*Recall/(Recall+Precision),2)


In [48]:
print(Precision,Accuracy,Recall,F1_Score)

73.64 71.43 86.17 79.41


In [49]:
clss=GradientBoostingClassifier()
clss.fit(x_train_scaled,y_train)
y_pred=clss.predict(x_test_scaled)
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test)*100)
Precision=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[0][1]),2)
Accuracy=round((confusion_matrix(y_pred,y_test)[0][0]
          +confusion_matrix(y_pred,y_test)[0][1])*100/sum(sum(confusion_matrix(y_pred,y_test))),2)
Recall=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[1][0]),2)

F1_Score=round(2*Precision*Recall/(Recall+Precision),2)
print(Precision,Accuracy,Recall,F1_Score)

              precision    recall  f1-score   support

           0       0.88      0.73      0.80       113
           1       0.50      0.73      0.59        41

    accuracy                           0.73       154
   macro avg       0.69      0.73      0.70       154
weighted avg       0.78      0.73      0.75       154

[[83 30]
 [11 30]]
73.37662337662337
73.45 73.38 88.3 80.19


In [50]:
clss=xb.XGBClassifier()
clss.fit(x_train_scaled,y_train)
y_pred=clss.predict(x_test_scaled)
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))
print(accuracy_score(y_pred,y_test)*100)
Precision=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[0][1]),2)
Accuracy=round((confusion_matrix(y_pred,y_test)[0][0]
          +confusion_matrix(y_pred,y_test)[0][1])*100/sum(sum(confusion_matrix(y_pred,y_test))),2)
Recall=round(confusion_matrix(y_pred,y_test)[0][0]*100/(confusion_matrix(y_pred,y_test)[0][0]
                                                 +confusion_matrix(y_pred,y_test)[1][0]),2)

F1_Score=round(2*Precision*Recall/(Recall+Precision),2)
print(Precision,Accuracy,Recall,F1_Score)

              precision    recall  f1-score   support

           0       0.87      0.73      0.79       113
           1       0.48      0.71      0.57        41

    accuracy                           0.72       154
   macro avg       0.68      0.72      0.68       154
weighted avg       0.77      0.72      0.73       154

[[82 31]
 [12 29]]
72.07792207792207
72.57 73.38 87.23 79.23


In [53]:
print("dnoe")

dnoe
